# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
****

In [11]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set()

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [5]:
# number of callbacks for black-sounding names
print(sum(data[data.race=='b'].call))

# number of callbacks for white-sounding names
print(sum(data[data.race=='w'].call))

157.0
235.0


In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 65 columns):
id                    4870 non-null object
ad                    4870 non-null object
education             4870 non-null int8
ofjobs                4870 non-null int8
yearsexp              4870 non-null int8
honors                4870 non-null int8
volunteer             4870 non-null int8
military              4870 non-null int8
empholes              4870 non-null int8
occupspecific         4870 non-null int16
occupbroad            4870 non-null int8
workinschool          4870 non-null int8
email                 4870 non-null int8
computerskills        4870 non-null int8
specialskills         4870 non-null int8
firstname             4870 non-null object
sex                   4870 non-null object
race                  4870 non-null object
h                     4870 non-null float32
l                     4870 non-null float32
call                  4870 non-null float32
city        

### 1) What test is appropriate for this problem? Does CLT apply?

Here we are prompted to determine whether race has a significant impact on the rate of callbacks.  A two-sample t-test seems to be the most appropriate test as we are going to compare two independent samples (race being a mutually exclusive factor) and that our sample size is large.  We can also infer that that Central Limit Theorem applies here as our sample size is large. 

In [32]:
# begin by subsetting relevant data and assigning variables
cbs_bl = data.call[data.race =='b']
cbs_wh = data.call[data.race =='w']

bl_samp_mean = np.mean(cbs_bl) #also serving as our callback rate/% of successes
wh_samp_mean = np.mean(cbs_wh)

bl_n = len(cbs_bl)
wh_n = len(cbs_wh)

print('The callback rate for resumes with black names is: ', bl_samp_mean)
print('The callback rate for resumes with white names is: ', wh_samp_mean)

print(bl_n, wh_n)

The callback rate for resumes with black names is:  0.0644763857126236
The callback rate for resumes with white names is:  0.09650924056768417
2435 2435


### 2) What are the null and alternate hypotheses?

An appropriate hypothesis test is as follows:

    *H0: There is no significant difference in callback rate (bl_samp_mean = wh_samp_mean)
    *Ha: There is a significant different in callback rates (bl_samp_mean ≠ wh_samp_mean)
    
We will assume a 95% confidence level for this test and thus reject the null hypothesis if the test statistic > 1.96 standard deviations and has a large p-value.

In [29]:
race_ttest, race_pval = stats.ttest_ind(a=cbs_bl, b=cbs_wh, equal_var=False)
print('The T-Statistic is %d with a p-value of %s' %(abs(race_ttest), race_pval))
print(race_ttest)

The T-Statistic is 4 with a p-value of 3.94294151365e-05
-4.11470529086


We see that our test statistic is 4.11 with a very small p-value.  Therefore, we reject the null hypothesis at the 95% confidence interval in favor of the alternative hypothesis.

In [31]:
# response to Question 3: Compute margin of error, confidence interval, and p-value
# p-value computed above

# margin of error / standard error
print('The standard error in the black name sample is: ', stats.sem(cbs_bl))
print('The standard error in the white name sample is: ', stats.sem(cbs_wh))

# confidence interval
conf_int_bl = stats.t.interval(alpha = .95, df = bl_n-1, loc = bl_samp_mean,
                 scale=stats.sem(cbs_bl))
print('At the 95% level, the lower bound of our confidence interval is ',conf_int_bl[0], 
      ' and the upper bound is ',conf_int_bl[1]) 

conf_int_wh = stats.t.interval(alpha = .95, df = wh_n-1, loc = wh_samp_mean,
                 scale=stats.sem(cbs_wh))
print('At the 95% level, the lower bound of our confidence interval is ',conf_int_wh[0], 
      ' and the upper bound is ',conf_int_wh[1]) 

The standard error in the black name sample is:  0.00497814343529
The standard error in the white name sample is:  0.0059853013975
At the 95% level, the lower bound of our confidence interval is  0.0547145496045  and the upper bound is  0.0742382218208
At the 95% level, the lower bound of our confidence interval is  0.0847724290314  and the upper bound is  0.108246052104


### 4) Write a story describing the statistical significance in the context of the original problem.

Given our sample data and our findings in this exercise, one can infer that a black- or white-sounding name on a resume has a significant effect on requests for interviews from employers.  We also see that resumes with a white-sounding name see a 50% higher callback rate over resumes with a black-sounding name.  Summary of the statistical inferences:
    
- Our hypothesis test indicates that we would  observe that the mean callback rates would be the same 4.11 standard deviations away.

- The standard error of both samples is ~0.5% which indicates their respective distributions are concentrated around the means.

- We also see there is no overlap in the confidence intervals for each sample.  This suggests that is very unlikely to see instances where the callback rate is the same for both samples.




### 5) Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

Not necessarily.  There are still over 62 columns worth exploring to see if there is any additional factor on callback rate.  It would be prudent to examine these other possible data points as they may be responsible for the significant difference in callback rate.  We explored the data in a manner that is framed by the assumption that race plays a factor in callback rate.  I think it would be prudent to expand the analysis over the entire dataset. With this analysis, we can proceed confirming the effect of race on callback rates or perhaps find other factors which affect callback rates (and that our finding that race is a factor would have been coincidence).  
